In [1]:
# -*- coding:utf-8 -*-
import pymongo

#### 抓取crawler上的MongoDb資料只針對2017的處理
client = pymongo.MongoClient('54.249.28.205',27017)

db = client['spadeAce']

### 讀入目前抓取的文章的資料
collection1 = db.awsmongoEttoday
collection2 = db.awsmongoNownews
collection3 = db.awsmongoLtn

allCollection = [collection1,collection2,collection3]
tmpList = []
tmpDict = {}
for collection in allCollection:
    news = collection.find({'Date':{'$gt':'2017'}})
    for oneNew in news:
        tmpDict = {}
        tmpDict['Content'] = oneNew['Content']
        tmpDict['Link'] = oneNew['Link']
        tmpDict['Title'] = oneNew['Title']
        tmpDict['Date'] = oneNew['Date']
        tmpList.append(tmpDict)
        print(oneNew['Date'])
        
client.close()

2017-03-11 21:06:00
2017-03-11 20:16:00
2017-03-11 20:01:00
2017-03-11 19:33:00
2017-03-11 19:05:00
2017-03-11 18:18:00
2017-03-11 18:08:00
2017-03-11 17:25:00
2017-03-11 16:59:00
2017-03-11 16:41:00
2017-03-11 22:54:00
2017-03-11 22:32:00
2017-03-11 22:06:00
2017-03-11 22:02:00
2017-03-11 21:42:00
2017-03-11 00:42:00
2017-03-10 21:42:00
2017-03-10 21:18:00
2017-03-11 11:12:00
2017-03-11 01:18:00
2017-03-11 15:33:00
2017-03-11 15:25:00
2017-03-11 14:41:00
2017-03-11 14:06:00
2017-03-11 13:48:00
2017-03-11 13:43:00
2017-03-11 13:38:00
2017-03-11 13:15:00
2017-03-11 13:00:00
2017-03-11 12:57:00
2017-03-11 12:31:00
2017-03-11 12:27:00
2017-03-11 12:17:00
2017-03-11 11:52:00
2017-03-11 11:43:00
2017-03-11 11:23:00
2017-03-11 11:06:00
2017-03-11 10:51:00
2017-03-11 10:47:00
2017-03-11 10:07:00
2017-03-11 10:04:00
2017-03-11 09:19:00
2017-03-11 09:04:00
2017-03-11 08:52:00
2017-03-11 08:23:00
2017-03-11 07:55:00
2017-03-11 07:24:00
2017-03-11 07:08:00
2017-03-11 07:00:00
2017-03-11 06:42:00


In [2]:

###  ====================== 後端測試區 - TF-IDF =========================
import json
import jieba
import re
import os  
import sys 
import time
from datetime import datetime
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  


allData = tmpList

jieba.set_dictionary('dict.txt-v2.big')    

### 載入停用字
print('Input Stop Words')
stopwordset = set()
with open('stopwords.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

print('Start Jieba Cut')
totalContent = []
### 從data中挑1000篇文章切詞
for i in range(0,len(allData)):
    article = ''
    content = allData[i]['Content']
    
    ### 斷詞, 產生一個結巴物件
    words = jieba.cut(content, cut_all=False)
    for word in words:   
        m = re.match(r'^[\u4E00-\u9FFFa-zA-Z]+$',word )
        if m is not None:
            if word not in stopwordset:
                article+=word
                article+=' '
    totalContent.append(article)

## 用來裝全部文章的權重值
textWeightList = []

## 用來裝前5權重值
tagList = []
print('Start Tf-Idf')

if __name__ == "__main__":  
    
    ## 該物件會將文本中的詞語轉換為詞頻矩陣，矩陣元素a [i] [j]表示j詞在i類文本下的詞頻 
    vectorizer=CountVectorizer()
    
    ## 該物件會話統計每個詞語的tf-idf權值 
    transformer=TfidfTransformer()
   
    ## 第一個fit_transform是計算tf-idf，第二個fit_transform是將文本轉成詞頻矩陣
    tfidf=transformer.fit_transform(vectorizer.fit_transform(totalContent))  
    
    ## 獲取詞袋模型中的所有詞語 
    word=vectorizer.get_feature_names()
    
    ## 將tf-idf矩陣抽取出來，元素a [i] [j]表示j詞在i類文本中的tf-idf權重 
    weight=tfidf.toarray()
    
    ## 打印每類文本的tf-idf詞語權重，第一個遍遍所有文本，第二個為便利某一類文本下的詞語權重
    for i in range(len(weight)):
        ## 用來裝單篇權重值
        textMining = {}
        ## print (u"-------这里输出第",i,u"类文本的词语tf-idf权重------"  )
        for j in range(len(word)):  
            ## print( word[j],weight[i][j] ) 
            textMining[word[j]] = weight[i][j]
        textWeightList.append(textMining)

print('Sorted the Weight of KeyWords')
### 取出文章，排序權重，取出前5以字典型式存取
for oneArticle in textWeightList:
    dict= sorted(oneArticle.items(), key=lambda d:d[1], reverse = True)

    #tag = {}
    #tag[dict[0][0]]=dict[0][1]
    #tag[dict[1][0]]=dict[1][1]
    #tag[dict[2][0]]=dict[2][1]
    #tag[dict[3][0]]=dict[3][1]
    #tag[dict[4][0]]=dict[4][1]
    #tagList.append(tag)
    
    ### 決定要放幾個tag
    tag = []
    tag.append(dict[0][0])
    tag.append(dict[1][0])
    tag.append(dict[2][0])
    tag.append(dict[3][0])
    tag.append(dict[4][0])
    tag.append(dict[5][0])
    tag.append(dict[6][0])
    tag.append(dict[7][0])
    tag.append(dict[8][0])
    tag.append(dict[9][0])
    tagList.append(tag)
        
### 載入要parse的資料
for i in range(len(tagList)):
    allData[i]['Tag'] =tagList[i]

pat = '[a-zA-Z{" "}{\.}{\,}{\-}a-zA-Z]+'

## 讀取檔案
listName = []

with open('nameList.txt','r',encoding='utf-8') as nl:
    for line in nl:
        name = line.split('\n')[0]
        listName.append(name.lower())

for i in range(0,len(tagList)):
    tagByName = []
    s = allData[i]['Content']
    v=re.findall(pat, s)
    for j in v:
        if j.lower() in listName:
            tagByName.append(j.lower())
    allData[i]['TagByName'] = tagByName
    

Building prefix dict from E:\projectSpadeForAWS\miningText\dict.txt-v2.big ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.uabf0f2a152495562962b777e0956322a.cache


Input Stop Words
Start Jieba Cut


Loading model cost 0.712 seconds.
Prefix dict has been built succesfully.


Start Tf-Idf
Sorted the Weight of KeyWords


In [3]:
###  ====================== 後端測試區 - 建立陣列裝要上傳的資料  =========================

### 資料寫進要上傳的陣列

updateList = []


for j in range(len(tagList)):
    updateList.append(allData[j])

print('Start Change the data format')    
### 時間格式改為 YYYYMMDD
for i in range(0,len(updateList)): 
    if allData[i]['Link'].startswith('http://www.nownews'):
        newTime = updateList[i]['Date'].split('T')[0].split('-')[0]+updateList[i]['Date'].split('T')[0].split('-')[1]+updateList[i]['Date'].split('T')[0].split('-')[2]
    if allData[i]['Link'].startswith('https://www.sportsv'):
        newTime = updateList[i]['Date'].split('/')[0]+updateList[i]['Date'].split('/')[1]+updateList[i]['Date'].split('/')[2]
    if allData[i]['Link'].startswith('http://sports.ettoday'):
        newTime = updateList[i]['Date'].split(' ')[0].split('-')[0]+updateList[i]['Date'].split(' ')[0].split('-')[1]+updateList[i]['Date'].split(' ')[0].split('-')[2]
    allData[i]['YearDate'] = newTime


Start Change the data format


In [4]:
###  ====================== 後端測試區 - 建立連線上傳資料  =========================

### 將上傳陣列的東西寫進資料庫
### 全部是一次傳
### 2017的版本是要一筆一筆傳

import pymongo
print('start Insert Data to MongoDB')

client = pymongo.MongoClient("54.249.28.205", 27017)
db = client['spadeAce']
collection = db.mlbNews
for i in updateList:
    if collection.find_one({'Link':i['Link']}) == None:
        collection.insert(i)
        print(i['Title'])
        print('data assess')
    else:
        print('data duplicated')

client.close()

start Insert Data to MongoDB


C:\Users\BIG DATA\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


經典賽／委內瑞拉力抗義大利求晉級　ET12日4點直播
data assess
經典賽／受徵召打WBC　盧戈訪波多黎各尋根
data assess
經典賽／德黑蘭領軍哥倫比亞　運彩開盤加拿大讓1.5分
data assess
經典賽／強壓賽揚巨投　波多黎各盧戈5.1局僅被敲1安
data assess
經典賽／加拿大、哥倫比亞拚存活　ET12日1點直播
data assess
經典賽／對戰美國四棒海　運彩開多明尼加讓2.5分
data assess
經典賽／東京八強開打　12日六場ETtoday全程接力直播
data assess
經典賽／死亡之組輸不得　委內瑞拉將背水一戰
data assess
經典賽／波多黎各新人王94猛　美技、全壘打1人包辦
data assess
經典賽／將代表義大利對決委國　費城菜鳥左投：太瘋狂
data assess
被痛批　棒協：現在已無公車費問題　組訓比以前好很多
data assess
經典賽／荷蘭范登賀克先發　小久保打線傷腦筋
data assess
SBL季後賽／我是最佳射手！　呂政儒單場6三分準翻
data assess
經典賽／想念中華隊友　林琨笙流淚：這個團隊超屌的
data assess
SBL季後賽／呂政儒神射6三分　裕隆暢飲台啤奪首勝
data assess
桃猿老董霸氣揭「拒徵召真相」：別怪我不借球員　網洗版推爆
data assess
經典賽／劉保佑堅持桃猿不徵召　棒協報帳只准搭公車
data assess
經典賽／桃猿拒徵召　劉保佑：棒協整碗端走不借球員
data assess
經典賽／接受美國徵召的理由　聽聽他們怎麼說
data assess
經典賽／最後一次終於圓夢了　陳用彩光榮褪下中國戰袍
data assess
經典賽／美首戰用6後援　米勒可能對決多明尼加
data assess
經典賽／亞洲首位大聯盟球員　村上雅則捐球棒給名人堂
data assess
經典賽／以色列每ㄧ勝都寫歷史　溫斯坦：自知實力在哪
data assess
三振朴炳鎬但也挨他轟　陳偉殷：球數落後時直球被轟
data assess
經典賽／單場3轟　波多黎各完封委內瑞拉
data assess
經典賽／4局41球完全比賽不得不退場　亞契與光芒有約定
data assess
經典賽／赫南德茲WBC最慘戰役　投不滿3局吞敗
data a